# <span style="color:magenta"> Prepare TE and gene model annotation files. </span>

This notebook combines the TE model data with the gene model data, as they were previously separated.

1. Get the annotations file (.gff3) and the reference file (ctg.fa)
2. SeqIO.parse through reference genome and get a list of all contig names.
3. Make pandas dataframe of annotation file.
4. Subset dataframe to column 0 that only has values in list of contig names from reference file.
5. Use "to.csv(\t)" to convert the subset to a tab-separated GFF file.
6. Cat this subset GFF file of TE data to the gene data GFF file.

In [17]:
import pybedtools
from pybedtools import BedTool
import os
import pysam
import pandas as pd
import glob
import wiggelen
import pybedtools
from pybedtools import BedTool
from Bio import SeqIO

In [28]:
#First we need to define the base dirs
DIRS ={}
DIRS['BASE'] =  '../../../../analysis/' # absolute path = '/home/anjuni/analysis/'
DIRS['GFF_INPUT'] = os.path.join(DIRS['BASE'], 'annotations')
DIRS['GFF_OUT'] = os.path.join(DIRS['BASE'], 'output')
DIRS['REF'] = '../../../../Pst_104_v13_assembly/'

In [29]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)
    else:
        print(value)

../../../../analysis/
../../../../analysis/annotations
../../../../analysis/output
../../../../Pst_104_v13_assembly/


In [27]:
#Make filepaths
gff_file_list = [fn for fn in glob.iglob('%s/*.gff3' % DIRS['GFF_INPUT'], recursive=True)]
ref_fh = os.path.join(DIRS['REF'], 'Pst_104E_v13_ph_ctg.fa')

In [24]:
#Check that the list works
print(*gff_file_list, sep='\n')

../../../../analysis/annotations/Pst_104E_v13_p_ctg.repeatgenesLT.gff3
../../../../analysis/annotations/Pst_104E_v13_h_ctg.repeatgenesLT.gff3
../../../../analysis/annotations/Pst_104E_v13_p_ctg.anno.gff3
../../../../analysis/annotations/Pst_104E_v13_h_ctg.anno.gff3


In [8]:
#Make a list of all contig names
for seq.id in  SeqIO.parse(ref_fh, 'fasta'):
    if seq.id #is not in the list:
    #append to the list
        SeqIO.write(seq, split_reference_fh, 'fasta')

5mC_hc_nanopolish_sorted


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5mC_hc_nanopolish_sorted
5mC_tombo_sorted
5mC_nanopolish_sorted
6mA_tombo_sorted
6mA_smrtlink_sorted
6mA_prob_smrtlink_sorted
